# Advanced Subtitle Generation Pipeline with Speaker Diarization

## AI Engineer Intern Assignment - Varvenkatesh

This notebook implements an agentic pipeline that generates accurate subtitles with speaker separation, designed to outperform Saarika-v2.5.

### Key Features:
- **Advanced Diarization**: Whisper + Pyannote.audio combination
- **AI Quality Check**: Ollama-powered quality assessment agent
- **Multiple Formats**: SRT, VTT, JSON, TXT outputs
- **Speaker Consistency**: Advanced alignment and post-processing

### Technical Approach:
1. **Multi-model Pipeline**: Combines Whisper (transcription) + Pyannote (diarization)
2. **Advanced Alignment**: Word-level timestamp alignment with speaker segments
3. **Quality Assessment**: AI agent evaluates output across multiple dimensions
4. **Post-processing**: Merging, filtering, and consistency checks


## Setup and Installation

In [ ]:
# Install required packages
!pip install -r requirements.txt

  Using cached pyannote.audio-3.3.2-py2.py3-none-any.whl.metadata (11 kB)
  Using cached yt_dlp-2025.7.21-py3-none-any.whl.metadata (175 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached ollama-0.5.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached webrtcvad-2.0.10.tar.gz (66 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached librosa-0.11.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached moviepy-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached asteroid_filterbanks-0.4.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached einops-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached lightning-2.5.2-py3-none-any.whl.metadata (38 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached 

In [1]:
# Import libraries
import sys
import os
import logging
from pathlib import Path
import json
import time

# Add src to path
sys.path.append('src')

from src.main_pipeline import SubtitlePipeline

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ Setup complete!")

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


✅ Setup complete!


## Pipeline Initialization

Initialize the pipeline with your preferred models. The pipeline uses:
- **Whisper**: For high-quality transcription
- **Pyannote**: For advanced speaker diarization
- **Ollama**: For AI-powered quality assessment

In [25]:
# Initialize pipeline
pipeline = SubtitlePipeline(
    whisper_model="medium",  # Options: tiny, base, small, medium, large
    ollama_model="llama3.1:latest",  # Your Ollama model
    output_dir="output",
    temp_dir="temp"
)

# Display pipeline info
# info = pipeline.get_pipeline_info()
'''print(f"Pipeline: {info['name']}")
print(f"Version: {info['version']}")
print(f"Features: {', '.join(info['features'])}")'''

INFO:src.main_pipeline:Initializing pipeline components...
INFO:modules.diarization_engine:Using device: cuda
INFO:modules.diarization_engine:Loading Whisper model: medium
INFO:modules.diarization_engine:Loading Pyannote diarization pipeline
INFO:httpx:HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
ERROR:modules.quality_agent:Failed to initialize Ollama client: 'name'


KeyError: 'name'

## Simple Test - Process YouTube Video

Let's test with the provided YouTube video example.

In [23]:
# Test with YouTube video
video_url = "https://youtu.be/zYJKq17GpEc?si=0apoU-vLWrmJfFox"

print(f"🎬 Processing video: {video_url}")
print("This may take a few minutes...")

# Process the video
results = pipeline.process_video_url(video_url, max_duration=300)  # 5 minutes max

# Display results
if results['success']:
    print("\n" + "="*60)
    print("🎉 SUCCESS!")
    print(f"⏱️  Processing time: {results['processing_time']:.1f}s")
    print(f"🎯 Quality score: {results['quality_report']['overall_score']:.2f}/10")
    print(f"📊 Confidence: {results['quality_report']['confidence_level']}")
    print(f"🎙️  Speakers: {results['speakers_count']}")
    print(f"📝 Segments: {results['segments_count']}")
    print("\n📁 Generated files:")
    for format_type, file_path in results['subtitle_files'].items():
        print(f"  - {format_type.upper()}: {file_path}")
    print("="*60)
else:
    print(f"❌ Failed: {results['error']}")

🎬 Processing video: https://youtu.be/zYJKq17GpEc?si=0apoU-vLWrmJfFox
This may take a few minutes...


NameError: name 'pipeline' is not defined

## Alternative Test - Local File

If you have a local video/audio file, you can test with this:

In [ ]:
# Test with local file (uncomment and modify path as needed)
# local_file = "path/to/your/video.mp4"  # Change this to your file path
# results = pipeline.process_local_video(local_file)

# Or for audio files:
# audio_file = "path/to/your/audio.wav"
# results = pipeline.process_audio_file(audio_file)

print("To test with local files, uncomment and modify the code above")

## View Generated Subtitles

Let's examine the generated subtitle files:

In [ ]:
# View SRT file (if generated)
output_dir = Path("output")
srt_files = list(output_dir.glob("*.srt"))

if srt_files:
    latest_srt = max(srt_files, key=os.path.getctime)
    print(f"📄 SRT File: {latest_srt.name}")
    print("\n" + "="*50)
    
    with open(latest_srt, 'r', encoding='utf-8') as f:
        content = f.read()
        # Show first 1000 characters
        print(content[:1000])
        if len(content) > 1000:
            print("\n... (truncated, see full file for complete content)")
    print("="*50)
else:
    print("No SRT files found. Run the processing cell above first.")

## Quality Assessment Report

View the AI quality assessment results:

In [ ]:
# View quality report
quality_files = list(output_dir.glob("*quality_report.json"))

if quality_files:
    latest_quality = max(quality_files, key=os.path.getctime)
    
    with open(latest_quality, 'r', encoding='utf-8') as f:
        quality_data = json.load(f)
    
    print("🔍 AI QUALITY ASSESSMENT REPORT")
    print("="*50)
    print(f"Overall Score: {quality_data['overall_score']:.2f}/10")
    print(f"Confidence Level: {quality_data['confidence_level']}")
    
    print("\n📊 Detailed Scores:")
    for metric, score in quality_data['detailed_scores'].items():
        print(f"  {metric.replace('_', ' ').title()}: {score:.2f}/10")
    
    print(f"\n📈 Metadata:")
    metadata = quality_data['metadata']
    print(f"  Total Segments: {metadata['total_segments']}")
    print(f"  Total Speakers: {metadata['total_speakers']}")
    print(f"  Duration: {metadata['total_duration']:.1f}s")
    print(f"  Avg Confidence: {metadata['avg_segment_confidence']:.2f}")
    
    if quality_data['issues']:
        print(f"\n⚠️  Issues Identified:")
        for issue in quality_data['issues'][:5]:  # Show first 5 issues
            print(f"  - {issue}")
    
    print(f"\n💡 Recommendations:")
    for rec in quality_data['recommendations']:
        print(f"  - {rec}")
    
    print("="*50)
else:
    print("No quality reports found. Run the processing cell above first.")

## Performance Comparison

### Why This Pipeline Outperforms Saarika-v2.5:

1. **Multi-Model Approach**: 
   - Whisper (OpenAI) for transcription
   - Pyannote.audio for diarization
   - Advanced alignment algorithms

2. **AI Quality Assessment**:
   - Real-time quality scoring
   - Multi-dimensional evaluation
   - Automated recommendations

3. **Advanced Post-Processing**:
   - Speaker consistency checks
   - Segment merging and filtering
   - Overlap resolution

4. **Robust Error Handling**:
   - Fallback mechanisms
   - Comprehensive logging
   - Graceful degradation

### Key Improvements:
- Better speaker boundary detection
- More consistent speaker labeling
- Higher transcription accuracy
- Reduced false speaker switches
- Comprehensive quality metrics

## Technical Documentation

### Pipeline Flow:

1. **Audio Extraction/Download**
   - YouTube video download via yt-dlp
   - Local file processing via moviepy
   - Audio preprocessing and normalization

2. **Diarization Process**
   - Whisper transcription with word-level timestamps
   - Pyannote speaker diarization
   - Advanced alignment algorithm
   - Post-processing and merging

3. **Quality Assessment**
   - AI-powered evaluation using Ollama
   - Multi-dimensional scoring
   - Issue identification and recommendations

4. **Output Generation**
   - SRT/VTT subtitle generation
   - JSON metadata export
   - Plain text transcript
   - Quality report

### Limitations:
- Requires good audio quality for optimal results
- Performance depends on number of speakers
- Processing time scales with audio length
- Requires Ollama server for quality assessment

### Future Improvements:
- Real-time processing capabilities
- Custom speaker model training
- Multi-language support enhancement
- GPU acceleration optimization
- Integration with cloud APIs

## Quick Test Function

Simple function to test the pipeline:

In [ ]:
def quick_test(input_source, max_duration=300):
    """
    Quick test function for the pipeline
    
    Args:
        input_source: URL or file path
        max_duration: Maximum duration in seconds
    """
    print(f"🚀 Quick Test: {input_source}")
    print("-" * 50)
    
    start_time = time.time()
    
    try:
        # Determine input type and process
        if input_source.startswith(('http://', 'https://')):
            results = pipeline.process_video_url(input_source, max_duration)
        elif Path(input_source).suffix.lower() in ['.mp3', '.wav', '.flac', '.m4a']:
            results = pipeline.process_audio_file(input_source)
        else:
            results = pipeline.process_local_video(input_source)
        
        # Print results
        if results['success']:
            print(f"✅ SUCCESS in {results['processing_time']:.1f}s")
            print(f"📊 Quality: {results['quality_report']['overall_score']:.2f}/10")
            print(f"🎙️  Speakers: {results['speakers_count']}")
            print(f"📝 Segments: {results['segments_count']}")
            print(f"📁 SRT: {results['subtitle_files']['srt']}")
        else:
            print(f"❌ FAILED: {results['error']}")
            
    except Exception as e:
        print(f"❌ ERROR: {e}")
    
    print("-" * 50)

# Example usage:
# quick_test("https://youtu.be/zYJKq17GpEc")
# quick_test("path/to/your/video.mp4")

print("Quick test function ready! Use quick_test('your_url_or_file') to test.")